In [3]:
from itertools import chain, combinations
import numpy as np  # Fundamental package for scientific computing with Python
import pandas as pd
import calendar
import time
from sklearn.metrics import accuracy_score, precision_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import sys
import os
from sklearn.tree import DecisionTreeClassifier
# Trova il percorso assoluto della cartella "esotic" (dove si trova il tuo ipynb)
current_dir = os.path.dirname(os.path.abspath("__file__"))

# Trova il percorso assoluto della cartella "lib"
lib_dir = os.path.join(current_dir, "..", "lib")

# Aggiungi il percorso di "lib" al PYTHONPATH
sys.path.append(lib_dir)

from Utility import *

In [4]:
dataframe = pd.DataFrame(columns=['DataSetName', 'TrainAccuracy', 'TrainPrecision', 'TrainF2', 'TrainRecall',
                                  'TrainROC', 'TestAccuracy', 'TestPrecision', 'TestF2', 'TestRecall', 'TestROC', "UnderTestAccuracy"
"OverTestAccuracy", 'BestParams'])

features = [
    'EloDiff', 
    "PointsDifference",
    "HomeGoalsRatio",
    "AwayGoalsRatio",
    "GoalsDifference",
    "ConcededGoalsDifference",
    'GoalsAvgSum',
    'RecentFormRatio',
    "EloRatio",
    "HomeLast3Points",
    "HomeAvgGoalsScored",
    "HomeAvgGoalsConceded",
    "HomeEwmaPoints",
    "HomeEwmaGoalsScored",
    "HomeEwmaGoalsConceded",
    "AwayLast3Points",
    "AwayAvgGoalsScored",
    "AwayAvgGoalsConceded",
    "AwayEwmaPoints",
    "AwayEwmaGoalsScored",
    "AwayEwmaGoalsConceded",
    "HomeWins",
    "HomeDraws",
    "HomeLosses",
    "AwayWins",
    "AwayDraws",
    "AwayLosses",
    "UltimoScontroDiretto",
    "Last3PointsDifference",
    "GoalRatioDifference",
    "EwmaGoalsSum",
    "GoalsSum",
    "HomePointGap",
    "AwayPointGap",
    "HomeGoalGap",
    "AwayGoalGap",
    "feat1",
    "feat2",
    'FormDiff',
    'RecentFormDiff', 
    'GoalCumulativeSum',
    # 'GoalCumulativeSumPrev'
     'ATW_LogRegPred'
]
classWeight={1: 1, 0: 2.5}
# features = ['AwayGoalsRatio', 'GoalsDifference', 'HomeEwmaGoalsScored']
combinazioni = chain.from_iterable(combinations(
    features, r) for r in range(1,3))#len(features) + 1))

combos = []

## Combo

In [5]:
for idx, combinazione in enumerate(combinazioni):
    current_GMT = time.gmtime()
    time_stamp = calendar.timegm(current_GMT)

    print(idx, time.strftime("%H:%M:%S", time.localtime()), ':', combinazione)
    combos.append(combinazione)

print('Length of combos',combos.__len__())

0 17:41:52 : ('EloDiff',)
1 17:41:52 : ('PointsDifference',)
2 17:41:52 : ('HomeGoalsRatio',)
3 17:41:52 : ('AwayGoalsRatio',)
4 17:41:52 : ('GoalsDifference',)
5 17:41:52 : ('ConcededGoalsDifference',)
6 17:41:52 : ('GoalsAvgSum',)
7 17:41:52 : ('RecentFormRatio',)
8 17:41:52 : ('EloRatio',)
9 17:41:52 : ('HomeLast3Points',)
10 17:41:52 : ('HomeAvgGoalsScored',)
11 17:41:52 : ('HomeAvgGoalsConceded',)
12 17:41:52 : ('HomeEwmaPoints',)
13 17:41:52 : ('HomeEwmaGoalsScored',)
14 17:41:52 : ('HomeEwmaGoalsConceded',)
15 17:41:52 : ('AwayLast3Points',)
16 17:41:52 : ('AwayAvgGoalsScored',)
17 17:41:52 : ('AwayAvgGoalsConceded',)
18 17:41:52 : ('AwayEwmaPoints',)
19 17:41:52 : ('AwayEwmaGoalsScored',)
20 17:41:52 : ('AwayEwmaGoalsConceded',)
21 17:41:52 : ('HomeWins',)
22 17:41:52 : ('HomeDraws',)
23 17:41:52 : ('HomeLosses',)
24 17:41:52 : ('AwayWins',)
25 17:41:52 : ('AwayDraws',)
26 17:41:52 : ('AwayLosses',)
27 17:41:52 : ('UltimoScontroDiretto',)
28 17:41:52 : ('Last3PointsDifference',

In [6]:
for idx, combo in enumerate(combos):
    current_GMT = time.gmtime()
    time_stamp = calendar.timegm(current_GMT)

    feat_cols = [col for col in combo]

    data = getCompleteATWData().copy()
    data = data.dropna(subset=feat_cols)
    data['isATS'] = np.where(((data['FTAG'] > 0)), 1, 0)

    # dataTrain = getTrainOver1Data().copy()
    # dataTest = getTestOver1Data().copy()
    # dataTrain = dataTrain.dropna(subset=feat_cols)
    # dataTest = dataTest.dropna(subset=feat_cols)
    # print(len(dataTrain), len(dataTest))


    # x_train = dataTrain[feat_cols]
    # y_train = dataTrain['isOver']
    # x_test = dataTest[feat_cols]
    # y_test = dataTest['isOver']
    x_train, x_test, y_train, y_test = train_test_split(
        data[feat_cols], data['isATS'], test_size=0.3, random_state=42, shuffle=True)#, stratify=data['isATS'])

    x_train = x_train.sort_index()
    x_test = x_test.sort_index()
    y_train = y_train.sort_index()
    y_test = y_test.sort_index()

    # Crea e addestra il modello di albero di decisione per la regressione
    # decTreeModel = DecisionTreeClassifier(random_state=42, min_samples_split=80
    #                              # , class_weight={1: 1.2, 0: 1}
    #                              # , class_weight='balanced'
    #                              , max_depth=3  # , ccp_alpha= 0.00001
    #                              # , max_features= 3
    #                              ).fit(x_train, y_train)
    
    # Crea e addestra il modello di LogRegression
    logRegModel = trainLogRegModel(x_train, y_train, class_weight=classWeight)

    # Crea e addestra il modello di Random Forest
    rfModel = RandomForestClassifier(random_state=42, n_estimators=100, 
                                min_samples_split=80, max_depth=3, class_weight=classWeight).fit(x_train, y_train) 

    # df_test = dataTest[dataTest.index.isin(x_test.index)].copy()
    # df_train = dataTrain[dataTrain.index.isin(x_train.index)].copy()
    df_test = data[data.index.isin(x_test.index)].copy()
    df_train = data[data.index.isin(x_train.index)].copy()
    # print(len(df_test), len(df_train))
    df_train['LogRegPred'] = logRegModel.predict(x_train)
    # df_train['DecTreePred'] = decTreeModel.predict(x_train)
    df_train['RFPred'] = rfModel.predict(x_train)

    df_test['LogRegPred'] = logRegModel.predict(x_test)
    # df_test['DecTreePred'] = decTreeModel.predict(x_test)
    df_test['RFPred'] = rfModel.predict(x_test)

    df_test['Quota'] = df_test['B365A'].fillna(df_test['AvgCA'].fillna(df_test['AvgA']))
    df_test['PA'] =  1/df_test['Quota']
    df_test['ATS'] = df_test['PA']**(1/4.8)#+0.003/df_test['PA']
    df_test['ATS'] = 1/df_test['ATS']
    
    # Ciclo sui modelli
    # models = ['LogReg', 'DecTree', 'RF']
    models = ['LogReg', 'RF']
    rows = []

    for model in models:
        df_test['Gain'+model] = df_test.apply(calculate_gain_ATS, axis=1, args=(model+'Pred',) )

        # df_test['Gain'] = df_test['GainOver'].loc(-1)+ df_test_under['GainUnder'].loc(-1) 
        # df_test['Equity'] = df_test['Gain'].cumsum()
        df_test['Equity'+model] = df_test['Gain'+model].cumsum()
        
        # Metrics
        train_accuracy = round(
            100*accuracy_score(y_train, df_train[model+'Pred']), 2)
        test_accuracy = round(
            100*accuracy_score(y_test, df_test[model+'Pred']), 2)
        
        # test_precision = precision_score(y_test, y_test_pred)
        test_f1 = f1_score(y_test, df_test[model+'Pred'])
        classAcc = class_accuracy(y_test, df_test[model+'Pred'])
        print(idx, time.strftime("%H:%M:%S", time.localtime()),classAcc['Over'], 'Trying with following combo', combo)
        
        # Save Results
        new_row = pd.Series({
            'Model': model,
            'Combo': str(combo),
            'TrainAccuracy': train_accuracy,
            'TestAccuracy': test_accuracy,
            'TestF1': test_f1,
            "UnderTestAccuracy": classAcc['Under'],
            "OverTestAccuracy": classAcc['Over'],
            'Equity'+model: df_test['Equity'+model].iloc[-1],
            # 'BestParams': model.get_params,

        })
        # print(new_row)
        dataframe = pd.concat(
            [dataframe, new_row.to_frame().T], ignore_index=True)

dataframe.to_excel("../Dataframe/"+str(time_stamp) + "ATSall.xlsx")

# #  print(result)

0 17:42:33 32.62 Trying with following combo ('EloDiff',)
0 17:42:33 32.62 Trying with following combo ('EloDiff',)


c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])
c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])
c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])
c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])


1 17:42:34 32.62 Trying with following combo ('PointsDifference',)
1 17:42:34 32.65 Trying with following combo ('PointsDifference',)


c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])
c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])


2 17:42:35 38.42 Trying with following combo ('HomeGoalsRatio',)
2 17:42:36 41.96 Trying with following combo ('HomeGoalsRatio',)
3 17:42:37 41.18 Trying with following combo ('AwayGoalsRatio',)
3 17:42:37 44.73 Trying with following combo ('AwayGoalsRatio',)
4 17:42:38 45.01 Trying with following combo ('GoalsDifference',)
4 17:42:38 46.12 Trying with following combo ('GoalsDifference',)
5 17:42:39 32.62 Trying with following combo ('ConcededGoalsDifference',)
5 17:42:39 32.62 Trying with following combo ('ConcededGoalsDifference',)


...

894 18:03:28 33.04 Trying with following combo ('feat2', 'RecentFormDiff')
894 18:03:28 33.11 Trying with following combo ('feat2', 'RecentFormDiff')


c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])
c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])


895 18:03:30 33.24 Trying with following combo ('feat2', 'GoalCumulativeSum')
895 18:03:30 33.14 Trying with following combo ('feat2', 'GoalCumulativeSum')
896 18:03:31 39.17 Trying with following combo ('feat2', 'ATW_LogRegPred')
896 18:03:31 40.64 Trying with following combo ('feat2', 'ATW_LogRegPred')
897 18:03:32 32.62 Trying with following combo ('FormDiff', 'RecentFormDiff')
897 18:03:32 33.1 Trying with following combo ('FormDiff', 'RecentFormDiff')


c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])
c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\BothToScore\..\lib\Utility.py:288: RuntimeWarning: invalid value encountered in scalar divide
  accuracy_class_1 = cm[1, 1] / (cm[0, 1] + cm[1, 1])


898 18:03:33 32.68 Trying with following combo ('FormDiff', 'GoalCumulativeSum')
898 18:03:33 33.08 Trying with following combo ('FormDiff', 'GoalCumulativeSum')
899 18:03:34 46.62 Trying with following combo ('FormDiff', 'ATW_LogRegPred')
899 18:03:35 35.64 Trying with following combo ('FormDiff', 'ATW_LogRegPred')
900 18:03:36 32.68 Trying with following combo ('RecentFormDiff', 'GoalCumulativeSum')
900 18:03:36 32.94 Trying with following combo ('RecentFormDiff', 'GoalCumulativeSum')
901 18:03:37 50.16 Trying with following combo ('RecentFormDiff', 'ATW_LogRegPred')
901 18:03:37 35.87 Trying with following combo ('RecentFormDiff', 'ATW_LogRegPred')
902 18:03:38 47.53 Trying with following combo ('GoalCumulativeSum', 'ATW_LogRegPred')
902 18:03:38 42.85 Trying with following combo ('GoalCumulativeSum', 'ATW_LogRegPred')


In [7]:
# dataframe.to_excel("../Dataframe/"+str(time_stamp) + "all.xlsx")